In [9]:
#Originally this code was called the long loop, but I thought it would be better to be more formal for the turned in title
#Either way, it is the longest loop I've ever made, and I find it incredible how fast a computer can calculate all of this data


#Power simulation BI
#This pandapower flow simulation calculates the peak loading on Las Islas Baleares at different times throughout the day. 
#Data for generation and demand were taken from REE's website on real time Balearic Islands demand
#This code is meant to run on Python 3 kernal

In [13]:
#import all the packages that will be needed in the following code
import pandas as pd
import pandapower as pp
from pandapower import plotting
import numpy as np
import matplotlib.pyplot as plt
import math
import numba

from scipy.signal import argrelextrema

In [14]:
# Starting with pandapower and setting all constant variables. 
#Create empty net
# https://pandapower.readthedocs.io/en/v2.4.0/elements/empty_network.html
net = pp.create_empty_network()

#initialize line voltages in MW assuming ibiza/formentera

hv220=220
hv132=132
#power factor calculations
#PF = 0.9, Q = P*tan(arccos(0.9))
#tan(arccos(0.9))=0.484322
PF= 0.484322


#initialize bus location for geodata epsg.4326 so we can later map this on QGIS

loc_eb=(3.1583316, 39.5822638)
loc_ll=(3.0399121, 39.6736644)
loc_sr=(2.6788739, 39.6511746)
loc_va=(2.5492521, 39.5843260)
loc_so=(2.7442129,39.6002680)
loc_mp=(3.092442,39.809435)
loc_gs=(3.42746869,39.73867947)

loc_cb = (3.83379024,39.93198792)
loc_ciut=(3.8553116,40.0033945)
loc_merc = (4.09591994,39.97708936)
loc_drag = (4.2366714,39.8912109)
loc_mahon = (4.25796449,39.89715096)

loc_ibiza=(1.4310005,38.9192119)

loc_sp=(2.50895716,39.53767893)

loc_cas = (2.7243168,39.5683692)



#Initialize line lengths 220 and 132 lines studied. All distances in KM and taken from from qgis

IB_SP=126 #https://www.ree.es/sites/default/files/01_ACTIVIDADES/Documentos/Romulo2_en.pdf

#mallorca line lengths 
SP_Val=6
Val_SR1=15
Val_SR2=15
SR_SO=9
CA_SO = 4
SR_LL=34
SO_LL=29
LL_MTR1=17
LL_MTR2=16
LL_EB1=15
LL_EB2=15
EB_GESA=30

#menorca line lengths
GESA_CB=41
CB_CIUT = 8
drag_mahon = 5
merc_drag = 15
ciut_drag = 36
ciut_merc = 21

In [15]:
#Creating all the buses for our powerflow
#create buses on mallorca

b_eb_lv = pp.create_bus(net, vn_kv=hv132, name = "Es Bessons LV", geodata=loc_eb)
b_eb_hv = pp.create_bus(net, vn_kv=hv220, name = "Es Bessons HV", geodata=loc_eb)

b_ll = pp.create_bus(net, vn_kv=hv220, name = "Lubi", geodata=loc_ll)
b_sr = pp.create_bus(net, vn_kv=hv220, name = "Son Rues", geodata=loc_sr)
b_va = pp.create_bus(net, vn_kv=hv220, name = "Valldurgent", geodata=loc_va)
b_so = pp.create_bus(net, vn_kv=hv220, name = "Son Orlandis", geodata=loc_so)
b_mp = pp.create_bus(net, vn_kv=hv220, name = "Murterar Power", geodata=loc_mp)
b_gs = pp.create_bus(net, vn_kv=hv132, name = "GESA", geodata=loc_gs)

b_cas = pp.create_bus(net, vn_kv=hv220, name = "Cas Tresorer", geodata=loc_cas)

#create buses for menorca (both modelled as load buses on 132kV network)
b_cb = pp.create_bus(net, vn_kv=hv132, name = "Cala en Bosc", geodata=loc_cb)
b_ciut = pp.create_bus(net, vn_kv=hv132, name = "Ciutadella", geodata=loc_ciut)
b_mahon = pp.create_bus(net, vn_kv=hv132, name = "Mahon", geodata=loc_mahon)
b_drag = pp.create_bus(net, vn_kv=hv132, name = "Dragonera SS", geodata=loc_drag)
b_merc = pp.create_bus(net, vn_kv=hv132, name = "Es Merceds SS", geodata=loc_merc)

#create bus representing ibiza/formentera (load bus on 132kV network)
b_ibiza = pp.create_bus(net, vn_kv=hv132, name = "Elvissa PS", geodata=loc_ibiza)

#create buses for mainland interconnector at Santa Ponsa

b_sp_lv = pp.create_bus(net, vn_kv=hv132, name = "SP HVDC LV", geodata=loc_sp) 
b_sp_hv = pp.create_bus(net, vn_kv=hv220, name = "SP HVDC HV", geodata=loc_sp)



In [16]:
#create lines and attach them to the nodes....
#create_line(net, “line1”, from_bus = 0, to_bus = 1, length_km=0.1, std_type=”NAYY 4x50 SE”)

#220 overhead typical properties....gull aw
Rkm_220OH = 0.13
Xkm_220OH = 0.425
Cnfkm_220OH = 8.2
MaxIkA220 = 0.8

#132 overhead properties....hawk aw
Rkm_132OH = 0.14
Xkm_132OH = 0.42
Cnfkm_132OH = 8.2
MaxIkA132 = 0.66

#132 Underwater/ground properties IBIZA MALLORCA
Rkm_132UW = 0.14
Xkm_132UW = 0.10
Cnfkm_132UW = 200
MaxIkA132UW = 1.6

#132 Underwater/ground properties Mallorca Menorca
Rkm_132UWmen = 0.14
Xkm_132UWmen = 0.10
Cnfkm_132UWmen = 200
MaxIkA132men = 1.6


In [17]:
#data taken from REE for June 27th, 2020
#data can be found on GITHUB

df = pd.read_csv ('station_demand.csv')
#print (df)

gen = pd.read_csv ('generation.csv')
#print (gen)


In [18]:
#Creating the list that all of our peak loading % will be put into. 
cols = ['Peak Wire Loading']
lst = []

In [19]:
#the name piece
# the long loop
#in this loop we are running a powerflow using all the variables outlined above. 
#The result should yield the peak loading % for each time of the day. 
n = 20
for n in range(20,144):
    p_eb=df.Es_Bessons[n]
    p_ll=df.Lubi[n]
    p_sr=df.Son_Rues[n]
    p_va=df.Valldurgent[n]
    p_so=df.Son_Orlandis[n]
    p_mp=df.Murterar_Power[n]
    p_gs=df.GESA[n]
    p_sp=df.Santa_Ponsa[n]
    p_cas=df.Cas_Tresoare[n]
#menorca load at 1320 is 53MW
    p_ciut= df.Ciutadella[n]
    p_cb = df.Cala_Bosc[n]
    p_mahon=df.Mahon[n]
    p_drag= df.Dragonera[n]
    p_merc=df.Es_Merceds[n]
#ibiza and formentera combined load minus generation at 1320. 
    p_ibiza=df.Elvissa[n]
    
#creating the loads
    L_eb = pp.create_load(net, bus=b_eb_lv, p_mw=p_eb, q_mvar=p_eb*PF, name = "Load Es Bessons")
    L_ll = pp.create_load(net, bus=b_ll, p_mw=p_ll, q_mvar=p_ll*PF, name = "Load Lubi")
    L_sr = pp.create_load(net, bus=b_sr, p_mw=p_sr, q_mvar=p_sr*PF, name = "Load Son Reus")
    L_va = pp.create_load(net, bus=b_va, p_mw=p_va, q_mvar=p_va*PF, name = "Load Valldurgent")
    L_so = pp.create_load(net, bus=b_so, p_mw=p_so, q_mvar=p_so*PF, name = "Load Son Orlandis")
    L_mp = pp.create_load(net, bus=b_mp, p_mw=p_mp, q_mvar=p_mp*PF, name = "Load Murtretar") 
    L_gs = pp.create_load(net, bus=b_gs, p_mw=p_gs, q_mvar=p_gs*PF, name = "Load GESA")
    L_sp = pp.create_load(net, bus=b_sp_lv, p_mw=p_sp, q_mvar=p_sp*PF, name = "Load Santa Ponsa")
    l_cas =pp.create_load(net, bus=b_cas, p_mw=p_cas, q_mvar=p_cas*PF, name = "Load Cas Tresoare")

    L_cb = pp.create_load(net, bus=b_cb, p_mw=p_cb, q_mvar=p_cb*PF, name = "Load Cala en bosc")
    L_ciut = pp.create_load(net, bus=b_ciut, p_mw=p_ciut, q_mvar=p_ciut*PF, name = "Load ciutadella")
    L_mahon = pp.create_load(net, bus=b_drag, p_mw=p_mahon, q_mvar=p_mahon*PF, name = "Load mahon")

    L_ibiza = pp.create_load(net, bus=b_ibiza, p_mw=p_ibiza, q_mvar=p_ibiza*PF, name = "Load Ibiza/Formentera")

    L_sp = pp.create_load(net, bus=b_sp_lv, p_mw=0, q_mvar=237, name = "Reactor")


#net.load.tail()

#define generation at 120pm on mallorca
    P_ccgt = gen.Combined_cycle[n]
#Combined cycle
    P_trash = gen.Wastes[n]
#waste
    P_cogen = gen.Cogeneration[n]
    P_solar = gen.Solar[n]

#define Menorca generation 32.6MW DIESEL 21.8MW GT (.1 wind, 1.1 solar, all into grid at mahon)
    P_MDG = gen.Other_renewables[n]+gen.Diesel_engines[n]+gen.Wind[n]

#create generator buses
#pandapower.create_sgen(net, bus, p_mw, q_mvar=0, sn_mva=nan, name=None)

    gen_sr = pp.create_sgen(net, bus = b_sr, p_mw = P_ccgt+P_trash+P_cogen, q_mvar=(P_ccgt+P_trash+P_cogen)*PF , name = "Son Reus GT and Waste")
    gen_solar = pp.create_sgen(net, bus = b_ll, p_mw = P_solar, q_mvar=0, name = "Lubi PV")
    gen_mahon = pp.create_sgen(net, bus = b_mahon, p_mw = P_MDG, q_mvar = P_MDG*PF, name = 'Mahon Diesel Gen')


#net.sgen.tail()

#create 220 to 132kV transformers at es bessons and santa ponsa, typical capacity zotero
    trans_sp = pp.create_transformer_from_parameters(net, hv_bus=b_sp_hv, lv_bus=b_sp_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400.*0.01, i0_percent=1)

    trans_eb = pp.create_transformer_from_parameters(net, hv_bus=b_eb_hv, lv_bus=b_eb_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400*0.01, i0_percent=1)


    swing_hvdc = pp.create_ext_grid(net, bus=b_sp_hv, vm_pu=1.00, name="Cometa Swing Bus")

    line_ibiza_sp1 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza , length_km= IB_SP , r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW, name = 'Ibiza_SP1', geodata = [loc_ibiza, loc_sp])
    line_ibiza_sp2 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza, length_km= IB_SP ,  r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW , name = 'Ibiza_SP2', geodata = [loc_ibiza, loc_sp])


#220kV network on mallorca
    line_sp_val1 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])
    line_sp_val2 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])

#two lines from santa ponsa to Valldurgent (sourced from overpass.eu)
    line_val_sr1 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR1 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='val_sr1', geodata = [loc_va, loc_sr])
    line_val_sr2 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR2 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'val_sr2', geodata = [loc_va, loc_sr])

    line_sr_so = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_so , length_km= SR_SO , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_so', geodata = [loc_sr, loc_so])
    line_cas_so1 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'cas_so1', geodata = [loc_cas, loc_so])
    line_cas_so2 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='cas_so2', geodata = [loc_cas, loc_so])

    line_sr_ll = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_ll , length_km= SR_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_ll', geodata = [loc_sr, loc_ll])
    line_so_ll = pp.create_line_from_parameters(net,  from_bus = b_so, to_bus = b_ll , length_km= SO_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'so_ll', geodata = [loc_so, loc_ll])

    line_LL_MP1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_mp1', geodata = [loc_ll, loc_mp])
    line_LL_MP2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'll_mp2', geodata = [loc_ll, loc_mp])

    line_LL_EB1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb1', geodata = [loc_ll, loc_eb])
    line_LL_EB2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb2', geodata = [loc_ll, loc_eb])


#132 lines on mallorca
    line_EB_GESA = pp.create_line_from_parameters(net, from_bus = b_eb_lv, to_bus = b_gs, length_km= EB_GESA,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name = 'eb_gesa', geodata = [loc_eb, loc_gs]) #change std type

    line_GESA_cb = pp.create_line_from_parameters(net, from_bus = b_gs, to_bus = b_cb, length_km= GESA_CB,  r_ohm_per_km=Rkm_132UWmen, x_ohm_per_km=Xkm_132UWmen, \
                                              c_nf_per_km = Cnfkm_132UWmen, max_i_ka =MaxIkA132men, name ='gesa_cb', geodata = [loc_gs, loc_cb]) #change std type

#standard 132 overhead lines
    line_cb_Ciut = pp.create_line_from_parameters(net, from_bus = b_cb, to_bus = b_ciut, length_km= CB_CIUT,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='cb_cuit', geodata = [loc_cb, loc_ciut])
    line_ciut_drag = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_drag, length_km= ciut_drag,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_drag', geodata = [loc_ciut, loc_drag])
    line_ciut_merc = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_merc, length_km= ciut_merc,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_merc',  geodata = [loc_ciut, loc_merc])
    line_merc_drag =  pp.create_line_from_parameters(net, from_bus = b_merc, to_bus = b_drag, length_km= merc_drag,   r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                 c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='merc_drag', geodata = [loc_merc, loc_drag] )
    line_drag_mahon1 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                    c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon1', geodata = [loc_drag, loc_mahon])
    line_drag_mahon2 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                  c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon2' , geodata = [loc_drag, loc_mahon])
#net.line.head()
    pp.runpp(net) 
    s=max(net.res_line.loading_percent)
    lst.append([s])
    n=n+1
    


In [20]:
#the name piece
n = 0
for n in range(0,144):
    p_eb=df.Es_Bessons[n]
    p_ll=df.Lubi[n]
    p_sr=df.Son_Rues[n]
    p_va=df.Valldurgent[n]
    p_so=df.Son_Orlandis[n]
    p_mp=df.Murterar_Power[n]
    p_gs=df.GESA[n]
    p_sp=df.Santa_Ponsa[n]
    p_cas=df.Cas_Tresoare[n]
#menorca load at 1320 is 53MW
    p_ciut= df.Ciutadella[n]
    p_cb = df.Cala_Bosc[n]
    p_mahon=df.Mahon[n]
    p_drag= df.Dragonera[n]
    p_merc=df.Es_Merceds[n]
#ibiza and formentera combined load minus generation at 1320. 
    p_ibiza=df.Elvissa[n]
    
#creating the loads
    L_eb = pp.create_load(net, bus=b_eb_lv, p_mw=p_eb, q_mvar=p_eb*PF, name = "Load Es Bessons")
    L_ll = pp.create_load(net, bus=b_ll, p_mw=p_ll, q_mvar=p_ll*PF, name = "Load Lubi")
    L_sr = pp.create_load(net, bus=b_sr, p_mw=p_sr, q_mvar=p_sr*PF, name = "Load Son Reus")
    L_va = pp.create_load(net, bus=b_va, p_mw=p_va, q_mvar=p_va*PF, name = "Load Valldurgent")
    L_so = pp.create_load(net, bus=b_so, p_mw=p_so, q_mvar=p_so*PF, name = "Load Son Orlandis")
    L_mp = pp.create_load(net, bus=b_mp, p_mw=p_mp, q_mvar=p_mp*PF, name = "Load Murtretar") 
    L_gs = pp.create_load(net, bus=b_gs, p_mw=p_gs, q_mvar=p_gs*PF, name = "Load GESA")
    L_sp = pp.create_load(net, bus=b_sp_lv, p_mw=p_sp, q_mvar=p_sp*PF, name = "Load Santa Ponsa")
    l_cas =pp.create_load(net, bus=b_cas, p_mw=p_cas, q_mvar=p_cas*PF, name = "Load Cas Tresoare")

    L_cb = pp.create_load(net, bus=b_cb, p_mw=p_cb, q_mvar=p_cb*PF, name = "Load Cala en bosc")
    L_ciut = pp.create_load(net, bus=b_ciut, p_mw=p_ciut, q_mvar=p_ciut*PF, name = "Load ciutadella")
    L_mahon = pp.create_load(net, bus=b_drag, p_mw=p_mahon, q_mvar=p_mahon*PF, name = "Load mahon")

    L_ibiza = pp.create_load(net, bus=b_ibiza, p_mw=p_ibiza, q_mvar=p_ibiza*PF, name = "Load Ibiza/Formentera")

    L_sp = pp.create_load(net, bus=b_sp_lv, p_mw=0, q_mvar=237, name = "Reactor")


#net.load.tail()

#define generation at 120pm on mallorca
    P_ccgt = gen.Combined_cycle[n]
#Combined cycle
    P_trash = gen.Wastes[n]
#waste
    P_cogen = gen.Cogeneration[n]
    P_solar = gen.Solar[n]

#define Menorca generation 32.6MW DIESEL 21.8MW GT (.1 wind, 1.1 solar, all into grid at mahon)
    P_MDG = gen.Other_renewables[n]+gen.Diesel_engines[n]+gen.Wind[n]

#create generator buses
#pandapower.create_sgen(net, bus, p_mw, q_mvar=0, sn_mva=nan, name=None)

#gen_cas = pp.create_sgen(net, bus = b_cas, p_mw = P_ccgt/2+P_cogen , q_mvar=  (P_ccgt/2+P_cogen)*PF , name = "Cas Tresoer Gas Turbine")
#gen_sr = pp.create_sgen(net, bus = b_sr, p_mw = P_ccgt/2+P_trash, q_mvar=(P_ccgt/2+P_trash)*PF , name = "Son Reus GT and Waste")

    gen_sr = pp.create_sgen(net, bus = b_sr, p_mw = P_ccgt+P_trash+P_cogen, q_mvar=(P_ccgt+P_trash+P_cogen)*PF , name = "Son Reus GT and Waste")
    gen_solar = pp.create_sgen(net, bus = b_ll, p_mw = P_solar, q_mvar=0, name = "Lubi PV")
    gen_mahon = pp.create_sgen(net, bus = b_mahon, p_mw = P_MDG, q_mvar = P_MDG*PF, name = 'Mahon Diesel Gen')


#gen_cas = pp.create_gen(net, bus = b_cas, p_mw = P_ccgt/2+P_cogen, vm_pu = 1.00, name = "Cas Tresoer Gas Turbine")
#gen_sr = pp.create_gen(net, bus = b_sr, p_mw = P_ccgt/2+P_trash, vm_pu = 1.00, name = "Son Reus GT and Waste")
#gen_solar = pp.create_gen(net, bus = b_ll, p_mw = P_solar, vm_pu = 1.00, name = "LLubi PV")
#gen_mahon = pp.create_gen(net, bus = b_mahon, p_mw = P_MDG, vm_pu = 1.00, name = 'Mahon Diesel Gen')


#net.sgen.tail()

#create 220 to 132kV transformers at es bessons and santa ponsa, typical capacity zotero
    trans_sp = pp.create_transformer_from_parameters(net, hv_bus=b_sp_hv, lv_bus=b_sp_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400.*0.01, i0_percent=1)

    trans_eb = pp.create_transformer_from_parameters(net, hv_bus=b_eb_hv, lv_bus=b_eb_lv, sn_mva=400, vn_hv_kv=220,\
                                                  vn_lv_kv=132, vkr_percent=1, vk_percent=10, pfe_kw=400*0.01, i0_percent=1)


    swing_hvdc = pp.create_ext_grid(net, bus=b_sp_hv, vm_pu=1.00, name="Cometa Swing Bus")


#pandapower.create_line_from_parameters(net, from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka, name=None, index=None, type=None, geodata=None, in_service=True, df=1.0, parallel=1, g_us_per_km=0.0, max_loading_percent=nan, alpha=None, temperature_degree_celsius=None, r0_ohm_per_km=nan, x0_ohm_per_km=nan, c0_nf_per_km=nan, g0_us_per_km=0, endtemp_degree=None, **kwargs)
#this needs to be updated to reflect underwater 132kV properties #CHANGE STD TYPE p = 2*100MW
#

    line_ibiza_sp1 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza , length_km= IB_SP , r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW, name = 'Ibiza_SP1', geodata = [loc_ibiza, loc_sp])
    line_ibiza_sp2 = pp.create_line_from_parameters(net, from_bus = b_sp_lv, to_bus = b_ibiza, length_km= IB_SP ,  r_ohm_per_km=Rkm_132UW, x_ohm_per_km=Xkm_132UW, \
                                                c_nf_per_km = Cnfkm_132UW, max_i_ka =MaxIkA132UW , name = 'Ibiza_SP2', geodata = [loc_ibiza, loc_sp])


#220kV network on mallorca
    line_sp_val1 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])
    line_sp_val2 = pp.create_line_from_parameters(net,  from_bus = b_sp_hv, to_bus = b_va , length_km= SP_Val , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'SP_Val', geodata = [loc_sp, loc_va])

#two lines from santa ponsa to Valldurgent (overpass.eu)
    line_val_sr1 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR1 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='val_sr1', geodata = [loc_va, loc_sr])
    line_val_sr2 = pp.create_line_from_parameters(net,  from_bus = b_va, to_bus = b_sr , length_km= Val_SR2 , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH,\
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'val_sr2', geodata = [loc_va, loc_sr])

    line_sr_so = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_so , length_km= SR_SO , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_so', geodata = [loc_sr, loc_so])
    line_cas_so1 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'cas_so1', geodata = [loc_cas, loc_so])
    line_cas_so2 = pp.create_line_from_parameters(net, from_bus = b_cas, to_bus = b_so, length_km= CA_SO, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                              c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='cas_so2', geodata = [loc_cas, loc_so])

    line_sr_ll = pp.create_line_from_parameters(net,  from_bus = b_sr, to_bus = b_ll , length_km= SR_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'sr_ll', geodata = [loc_sr, loc_ll])
    line_so_ll = pp.create_line_from_parameters(net,  from_bus = b_so, to_bus = b_ll , length_km= SO_LL , r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                            c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'so_ll', geodata = [loc_so, loc_ll])


    line_LL_MP1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_mp1', geodata = [loc_ll, loc_mp])
    line_LL_MP2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_mp, length_km= LL_MTR2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name = 'll_mp2', geodata = [loc_ll, loc_mp])

    line_LL_EB1 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB1, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb1', geodata = [loc_ll, loc_eb])
    line_LL_EB2 = pp.create_line_from_parameters(net, from_bus = b_ll, to_bus = b_eb_hv, length_km= LL_EB2, r_ohm_per_km=Rkm_220OH, x_ohm_per_km=Xkm_220OH, \
                                             c_nf_per_km = Cnfkm_220OH, max_i_ka =MaxIkA220, name ='ll_eb2', geodata = [loc_ll, loc_eb])


#132 lines on mallorca
    line_EB_GESA = pp.create_line_from_parameters(net, from_bus = b_eb_lv, to_bus = b_gs, length_km= EB_GESA,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name = 'eb_gesa', geodata = [loc_eb, loc_gs]) #change std type


#this needs to be updated to reflect underwater 132kV properties
    line_GESA_cb = pp.create_line_from_parameters(net, from_bus = b_gs, to_bus = b_cb, length_km= GESA_CB,  r_ohm_per_km=Rkm_132UWmen, x_ohm_per_km=Xkm_132UWmen, \
                                              c_nf_per_km = Cnfkm_132UWmen, max_i_ka =MaxIkA132men, name ='gesa_cb', geodata = [loc_gs, loc_cb]) #change std type

#standard 132 overhead lines
    line_cb_Ciut = pp.create_line_from_parameters(net, from_bus = b_cb, to_bus = b_ciut, length_km= CB_CIUT,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                              c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='cb_cuit', geodata = [loc_cb, loc_ciut])
    line_ciut_drag = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_drag, length_km= ciut_drag,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_drag', geodata = [loc_ciut, loc_drag])
    line_ciut_merc = pp.create_line_from_parameters(net, from_bus = b_ciut, to_bus = b_merc, length_km= ciut_merc,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='ciut_merc',  geodata = [loc_ciut, loc_merc])
    line_merc_drag =  pp.create_line_from_parameters(net, from_bus = b_merc, to_bus = b_drag, length_km= merc_drag,   r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                 c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='merc_drag', geodata = [loc_merc, loc_drag] )
    line_drag_mahon1 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH,\
                                                    c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon1', geodata = [loc_drag, loc_mahon])
    line_drag_mahon2 = pp.create_line_from_parameters(net, from_bus = b_drag, to_bus = b_mahon, length_km= drag_mahon,  r_ohm_per_km=Rkm_132OH, x_ohm_per_km=Xkm_132OH, \
                                                  c_nf_per_km = Cnfkm_132OH, max_i_ka =MaxIkA132, name ='drag_mahon2' , geodata = [loc_drag, loc_mahon])
#net.line.head()
    pp.runpp(net) 
    s=max(net.res_line.loading_percent)
    lst.append([s])
    n=n+1
#If desired view the list of peak loading %
#print(lst)

In [21]:
stress = pd.DataFrame(lst, columns=cols)
print(stress)

stress.to_excel("workwork.xlsx")  


     Peak Wire Loading
0            44.386434
1            44.202199
2            44.058145
3            43.897419
4            43.776215
..                 ...
263          60.690453
264          60.701925
265          60.704601
266          60.708525
267          60.706931

[268 rows x 1 columns]


In [ ]:
#view last pp results
#pp.runpp(net)
#net.res_line

In [24]:
#adding time to this dataframe
stress['Hora']=gen.Hora
stress.head()

#the excel sheet with this dataframe can now be analyzed to determine the hourly loadshape of peak loading % and the times of peak loading 

SyntaxError: invalid syntax (<ipython-input-24-c1bf749b5feb>, line 2)

In [125]:
print(max(net.res_line.loading_percent))

58.73936692783457
